# ibm_sql_query

Execute arbitrary SQL queries against CSV and PARQUET files using IBM Cloud SQL Query and Cloud Object Store

In [ ]:
#!pip uninstall -y ibm-cos-sdk ibm-cos-sdk-core ibmcloudsql ibm-watson-machine-learning ibm-watson-studio-pipelines 
!pip install nbformat numpy ibm-cos-sdk-core ibm-cos-sdk ibm-watson-machine-learning ibm-watson-studio-pipelines ibmcloudsql
docker_file="""
FROM registry.access.redhat.com/ubi8/python-39
RUN pip install ipython nbformat numpy ibm-cos-sdk-core ibm-cos-sdk ibm-watson-machine-learning ibm-watson-studio-pipelines ibmcloudsql
RUN wget https://raw.githubusercontent.com/IBM/claimed/master/component-library/transform/ibm-sql-query-cpd.ipynb
"""

In [ ]:
import glob
import logging
import ibmcloudsql
from ibmcloudsql import SQLQuery
import os
import shutil
import sys
import re
from ibm_watson_machine_learning import APIClient
from ibm_watson_studio_pipelines import WSPipelines
from ibm_watson_studio_pipelines.cpd_paths import CpdScope, CpdPath

In [ ]:
# target dir_path
target_dir_path = os.environ.get('target_dir_path')

# target asset name
target_asset_name = os.environ.get('target_asset_name')

# sql statement to execute ()
sql = os.environ.get('sql')

# IBM Cloud Token (alternative to API key)
token = os.environ.get('token')

# (unique) Custom Resource Name (CRN) of IBM SQL Query Service
sql_query_crn = os.environ.get('sql_query_crn')

# default: CSV - (will be generated into according STORED AS … clause in the INTO clause)
out_format = os.environ.get('out_format' , 'CSV')

# if set - will be generated into according PARTITIONED BY (<columns>) clause in the INTO clause)
out_partition_columns = os.environ.get('out_partition_columns')

# will be generated into according PARTITIONED INTO <num> OBJECTS clause in INTO clause
out_number_of_objects = os.environ.get('out_number_of_objects')

# will be generated into according PARTITIONED EVERY <num> ROWS clause in INTO clause
out_rows_per_object = os.environ.get('out_rows_per_object')

# default: False - only valid when no partitioning option is specified. Will be generated into sqlClient.rename_exact_result(jobid) after SQL has run.
out_exact_name = bool(os.environ.get('out_exact_name', False))

# default: False - will be generated into JOBPREFIX NONE in the INTO clause. Will cause results of previous runs with same output_uri to be overwritten, because no unique sub folder will be created for the result)
out_no_jobid_folder = bool(os.environ.get('out_no_jobid_folder', False))


In [ ]:
#token = 'eyJraWQiOiIyMDIyMDExNjA4MjIiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC0yNzAwMDI1NzNZIiwiaWQiOiJJQk1pZC0yNzAwMDI1NzNZIiwicmVhbG1pZCI6IklCTWlkIiwic2Vzc2lvbl9pZCI6IkMtMzJiYTVkMGQtODFlMi00NDk5LThiZjgtMjc5MGQzNmVjNjZkIiwianRpIjoiNDkyYjMxNDMtYTA4OC00MDRmLTlhY2YtYzJkMjcyZWY3ZjcwIiwiaWRlbnRpZmllciI6IjI3MDAwMjU3M1kiLCJnaXZlbl9uYW1lIjoiUm9tZW8iLCJmYW1pbHlfbmFtZSI6IktpZW56bGVyIiwibmFtZSI6IlJvbWVvIEtpZW56bGVyIiwiZW1haWwiOiJyb21lby5raWVuemxlckBjaC5pYm0uY29tIiwic3ViIjoicm9tZW8ua2llbnpsZXJAY2guaWJtLmNvbSIsImF1dGhuIjp7InN1YiI6InJvbWVvLmtpZW56bGVyQGNoLmlibS5jb20iLCJpYW1faWQiOiJJQk1pZC0yNzAwMDI1NzNZIiwibmFtZSI6IlJvbWVvIEtpZW56bGVyIiwiZ2l2ZW5fbmFtZSI6IlJvbWVvIiwiZmFtaWx5X25hbWUiOiJLaWVuemxlciIsImVtYWlsIjoicm9tZW8ua2llbnpsZXJAY2guaWJtLmNvbSJ9LCJhY2NvdW50Ijp7ImJvdW5kYXJ5IjoiZ2xvYmFsIiwidmFsaWQiOnRydWUsImJzcyI6IjliMTNiODU3YTMyMzQxYjcxNjcyNTVkZTcxNzE3MmY1IiwiaW1zX3VzZXJfaWQiOiI4MDgwMjIyIiwiaW1zIjoiMjAzMjQ3NCJ9LCJpYXQiOjE2NDMzOTU1ODksImV4cCI6MTY0MzM5Njc4OSwiaXNzIjoiaHR0cHM6Ly9pYW0uY2xvdWQuaWJtLmNvbS9pZGVudGl0eSIsImdyYW50X3R5cGUiOiJ1cm46aWJtOnBhcmFtczpvYXV0aDpncmFudC10eXBlOnBhc3Njb2RlIiwic2NvcGUiOiJpYm0gb3BlbmlkIiwiY2xpZW50X2lkIjoiYngiLCJhY3IiOjEsImFtciI6WyJwd2QiXX0.erONYtjfyZY9UGQ8yXWMk30VE9zDfmhJdoZ6bKCQY_p7vjeXZJT2OyzzzKGzQGz0ftWWhmjOdhexoE2vsLoKKs3LrXhY7_mnp9D5nPg9CXKH-0soXmJwOeSIn3nBWDnJCyVdUhuXiOzETEF_IfBwzgdhB3h5G6MtssRqxw_W4ra_E_hl-blsAAvNZfVbictfc9kXSzZ7tFsKMeMU1BD9prOVDpAjAwY4wijF_wLKPEJFDrpHmHrgF2lYsileJ0nMv7f6P-MreCTyMxAssJ56EO9ZcRR0vIeQa5YzzMWTEBtDtceYpUSymBRgovgmqD3OgHye0Y2CvE_1D0skGAw0Xw'
#cos_location='cpd:///projects/e0bce158-a9e4-4be6-a1da-20a04a7770f5/connections/de544f56-05c0-4dda-9e00-06458052c467/files/cos-rkie-sql-engine-test/sql_results'
#sql='SELECT * FROM cos://eu-de/claimed-test/data.parquet stored as parquet'
#sql_query_crn='crn:v1:bluemix:public:sql-query:us-south:a/9b13b857a32341b7167255de717172f5:f9dd6c9e-b24b-4506-819e-e038c92339e4::'
#target_asset_name='target_asset_name'

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

out_exact_name = bool(out_exact_name)
out_no_jobid_folder = bool(out_no_jobid_folder)


for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
cpdaas_url = 'https://api.dataplatform.cloud.ibm.com'

In [ ]:
cos_location_path = CpdPath.from_string(cos_location)
project_id = cos_location_path.scope_id()
connection_id = cos_location_path.resource_id()
bucket_name = cos_location_path.bucket_name()
dir_path = cos_location_path.file_path()

In [ ]:
pipelines_client = WSPipelines.from_token(token, url=cpdaas_url)
wml_credentials = pipelines_client.get_wml_credentials(cos_location_path.scope())

In [ ]:
wml_credentials['token'] = token
wml_client = APIClient(wml_credentials)
wml_client.set.default_project(project_id)

In [ ]:
wml_client.connections.get_details(connection_id)

In [ ]:
cos_props = wml_client.connections.get_details(connection_id)
cos_props

In [ ]:
cos_props = cos_props['entity']['properties']
access_key = cos_props['access_key']
secret_key = cos_props['secret_key']
url = cos_props['url'].split('//')[1]
bucket = cos_props['bucket']
target_location = "cos://{}/{}{}".format(url,bucket, dir_path)
target_location

In [ ]:
sqlClient = SQLQuery(api_key=None, token=token, instance_crn=sql_query_crn)

sql = sql + ' INTO {}'.format(target_location)

partitioned_by = False

if out_partition_columns is not None and len(out_partition_columns) > 0:
    if not partitioned_by:
        sql = sql + ' PARTITIONED'
        partitioned_by = True
    sql = sql + ' BY ({})'.format(out_partition_columns)

if out_number_of_objects is not None and len(out_number_of_objects) > 0:
    if not partitioned_by:
        sql = sql + ' PARTITIONED'
        partitioned_by = True
    sql = sql + ' INTO {} OBJECTS'.format(out_number_of_objects)    

if out_rows_per_object is not None and len(out_rows_per_object) > 0:
    if not partitioned_by:
        sql = sql + ' PARTITIONED'
        partitioned_by = True
    sql = sql + ' EVERY {} ROWS'.format(out_rows_per_object)

if out_no_jobid_folder:
    sql = sql + ' JOBPREFIX NONE'
    
sql = sql + ' STORED AS {}'.format(out_format)

if out_exact_name:
    job_id = sqlClient.submit_sql(sql)
    job_status = sqlClient.wait_for_job(job_id)
    print("Job " + job_id + " terminated with status: " + job_status)
    sqlClient.rename_exact_result(job_id) 
else:
    sqlClient.run_sql(sql)

In [ ]:
print(sql)

In [ ]:
metadata = {
  wml_client.data_assets.ConfigurationMetaNames.NAME: target_asset_name,
  wml_client.data_assets.ConfigurationMetaNames.DESCRIPTION: 'Data asset created for SQL Query result',
  wml_client.data_assets.ConfigurationMetaNames.CONNECTION_ID: connection_id,
  wml_client.data_assets.ConfigurationMetaNames.DATA_CONTENT_NAME: dir_path
}
asset_details = wml_client.data_assets.store(meta_props=metadata)
asset_details